<a href="https://colab.research.google.com/github/mlfa03/Data-Science-Projects/blob/main/artist_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artist recommender system with Pyspark 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 73kB/s 
     |████████████████████████████████| 204kB 42.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=c93c9f0a427907d1493792a1884d98a1addff5b3bd4d9314a546c77e7a9d3579
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
import tarfile

In [ ]:
spark = SparkSession.builder.appName("lastfm").getOrCreate()

## Getting to know the dataset

In [ ]:
!wget http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz


--2020-12-28 11:38:25--  http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz
Resolving mtg.upf.edu (mtg.upf.edu)... 84.89.139.55
Connecting to mtg.upf.edu (mtg.upf.edu)|84.89.139.55|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 569202935 (543M) [application/x-gzip]
Saving to: ‘lastfm-dataset-360K.tar.gz’

lastfm-dataset-360K 100%[===================>] 542.83M  1.24MB/s    in 8m 59s  

2020-12-28 11:47:24 (1.01 MB/s) - ‘lastfm-dataset-360K.tar.gz’ saved [569202935/569202935]



In [ ]:
filename = 'lastfm-dataset-360K.tar.gz'
tf = tarfile.open(filename)
print(tf.getnames())

['lastfm-dataset-360K', 'lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv', 'lastfm-dataset-360K/README.txt', 'lastfm-dataset-360K/mbox_sha1sum.py', 'lastfm-dataset-360K/usersha1-profile.tsv']


In [ ]:
file_plays = '/content/drive/MyDrive/Colab Notebooks/Lastfm/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'

In [ ]:
file_users = '/content/drive/MyDrive/Colab Notebooks/Lastfm/lastfm-dataset-360K/usersha1-profile.tsv'

#### Reading the data on listenings 


In [ ]:
df_plays = spark.read.csv(file_plays, sep="\t")
df_plays.show()

+--------------------+--------------------+--------------------+----+
|                 _c0|                 _c1|                 _c2| _c3|
+--------------------+--------------------+--------------------+----+
|00000c289a1829a80...|3bd73256-3905-4f3...|     betty blowtorch|2137|
|00000c289a1829a80...|f2fb0ff0-5679-42e...|           die Ärzte|1099|
|00000c289a1829a80...|b3ae82c2-e60b-455...|   melissa etheridge| 897|
|00000c289a1829a80...|3d6bbeb7-f90e-4d1...|           elvenking| 717|
|00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706|
|00000c289a1829a80...|8bfac288-ccc5-448...|red hot chili pep...| 691|
|00000c289a1829a80...|6531c8b1-76ea-414...|              magica| 545|
|00000c289a1829a80...|21f3573f-10cf-44b...|the black dahlia ...| 507|
|00000c289a1829a80...|c5db90c4-580d-4f3...|         the murmurs| 424|
|00000c289a1829a80...|0639533a-0402-40b...|          lunachicks| 403|
|00000c289a1829a80...|a342964d-ca53-4e5...|    walls of jericho| 393|
|00000c289a1829a80..

Renaming the columns to make it easier the analysis:
source code: https://gist.github.com/colbyford/da0d8a22eb6200ffcb9658983537d0e3

In [ ]:
column_list = df_plays.columns
#prefix = "my_prefix"
new_column_list = ["User_id", "Artist_id", "Artist_name", "Plays_by_user"]
 
column_mapping = [[o, n] for o, n in zip(column_list, new_column_list)]

df_plays = df_plays.select(list(map(lambda old, new: col(old).alias(new),*zip(*column_mapping))))

In [ ]:
df_plays.show()

+--------------------+--------------------+--------------------+-------------+
|             User_id|           Artist_id|         Artist_name|Plays_by_user|
+--------------------+--------------------+--------------------+-------------+
|00000c289a1829a80...|3bd73256-3905-4f3...|     betty blowtorch|         2137|
|00000c289a1829a80...|f2fb0ff0-5679-42e...|           die Ärzte|         1099|
|00000c289a1829a80...|b3ae82c2-e60b-455...|   melissa etheridge|          897|
|00000c289a1829a80...|3d6bbeb7-f90e-4d1...|           elvenking|          717|
|00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks|          706|
|00000c289a1829a80...|8bfac288-ccc5-448...|red hot chili pep...|          691|
|00000c289a1829a80...|6531c8b1-76ea-414...|              magica|          545|
|00000c289a1829a80...|21f3573f-10cf-44b...|the black dahlia ...|          507|
|00000c289a1829a80...|c5db90c4-580d-4f3...|         the murmurs|          424|
|00000c289a1829a80...|0639533a-0402-40b...|         

#### Reading users dataframe:
usersha1-profile.tsv:
      user-mboxsha1 \t gender ('m'|'f'|empty) \t age (int|empty) \t country (str|empty) \t signup (date|empty)

In [ ]:
df_users = spark.read.csv(file_users, sep="\t")
df_users.show()

+--------------------+----+----+--------------+------------+
|                 _c0| _c1| _c2|           _c3|         _c4|
+--------------------+----+----+--------------+------------+
|00000c289a1829a80...|   f|  22|       Germany| Feb 1, 2007|
|00001411dc427966b...|   f|null|        Canada| Dec 4, 2007|
|00004d2ac9316e22d...|null|null|       Germany| Sep 1, 2006|
|000063d3fe1cf2ba2...|   m|  19|        Mexico|Apr 28, 2008|
|00007a47085b9aab8...|   m|  28| United States|Jan 27, 2006|
|0000c176103e538d5...|   m|  20|United Kingdom|Jan 14, 2006|
|0000ee7dd906373ef...|   m|  17|       Finland|Nov 17, 2007|
|0000ef373bbd0d89c...|   f|  22|        Poland|May 23, 2007|
|0000f687d4fe9c1ed...|   f|  24|         Spain| Nov 4, 2008|
|0001399387da41d55...|   m|null|       Ukraine|Aug 17, 2008|
|000163263d2a41a39...|   m|  27|        Sweden| Jan 5, 2007|
|0001a57568309b287...|null|  23| United States|May 12, 2007|
|0001a88a7092846ab...|null|null|         Japan|Oct 13, 2008|
|0001bd96207f323b5...|  

In [ ]:
column_list = df_users.columns
#prefix = "my_prefix"
new_column_list = ["User_id","Gender", "Age", "Country", "Date"]
 
column_mapping = [[o, n] for o, n in zip(column_list, new_column_list)]

df_users = df_users.select(list(map(lambda old, new: col(old).alias(new),*zip(*column_mapping))))

In [ ]:
df_users.show()

+--------------------+------+----+--------------+------------+
|             User_id|Gender| Age|       Country|        Date|
+--------------------+------+----+--------------+------------+
|00000c289a1829a80...|     f|  22|       Germany| Feb 1, 2007|
|00001411dc427966b...|     f|null|        Canada| Dec 4, 2007|
|00004d2ac9316e22d...|  null|null|       Germany| Sep 1, 2006|
|000063d3fe1cf2ba2...|     m|  19|        Mexico|Apr 28, 2008|
|00007a47085b9aab8...|     m|  28| United States|Jan 27, 2006|
|0000c176103e538d5...|     m|  20|United Kingdom|Jan 14, 2006|
|0000ee7dd906373ef...|     m|  17|       Finland|Nov 17, 2007|
|0000ef373bbd0d89c...|     f|  22|        Poland|May 23, 2007|
|0000f687d4fe9c1ed...|     f|  24|         Spain| Nov 4, 2008|
|0001399387da41d55...|     m|null|       Ukraine|Aug 17, 2008|
|000163263d2a41a39...|     m|  27|        Sweden| Jan 5, 2007|
|0001a57568309b287...|  null|  23| United States|May 12, 2007|
|0001a88a7092846ab...|  null|null|         Japan|Oct 13

In [ ]:
distinct_ids = [x.Country for x in df_users.select('Country').distinct().collect()]

In [ ]:
len(distinct_ids)

239

## Cleaning the data

#### Listenings Data

In [ ]:
df_plays = df_plays.drop('Artist_id')
df_plays.show()

+--------------------+--------------------+-------------+
|             User_id|         Artist_name|Plays_by_user|
+--------------------+--------------------+-------------+
|00000c289a1829a80...|     betty blowtorch|         2137|
|00000c289a1829a80...|           die Ärzte|         1099|
|00000c289a1829a80...|   melissa etheridge|          897|
|00000c289a1829a80...|           elvenking|          717|
|00000c289a1829a80...|juliette & the licks|          706|
|00000c289a1829a80...|red hot chili pep...|          691|
|00000c289a1829a80...|              magica|          545|
|00000c289a1829a80...|the black dahlia ...|          507|
|00000c289a1829a80...|         the murmurs|          424|
|00000c289a1829a80...|          lunachicks|          403|
|00000c289a1829a80...|    walls of jericho|          393|
|00000c289a1829a80...|      letzte instanz|          387|
|00000c289a1829a80...|           goldfrapp|          361|
|00000c289a1829a80...|          horrorpops|          358|
|00000c289a182

In [ ]:
df_plays = df_plays.na.drop()

In [ ]:
row_numbers = df_plays.count()
column_numbers = len(df_plays.columns)

In [ ]:
print(row_numbers, column_numbers)

12363697 3


#### Users Data

In [ ]:
df_users = df_users.na.drop()

In [ ]:
row_numbers2 = df_users.count()
column_numbers2 = len(df_users.columns)
print(row_numbers2, column_numbers2)

267383 5


## Preparing the data for the ML Model

In [ ]:
indexer = [StringIndexer(inputCol=col, outputCol=col+'_index').fit(df_plays) for col in list(set(df_plays.columns)- set(['count']))
]
pipeline = Pipeline(stages=indexer)

data = pipeline.fit(df_plays).transform(df_plays)
data.show()

+--------------------+--------------------+-------------+-------------------+-------------+-----------------+
|             User_id|         Artist_name|Plays_by_user|Plays_by_user_index|User_id_index|Artist_name_index|
+--------------------+--------------------+-------------+-------------------+-------------+-----------------+
|00000c289a1829a80...|     betty blowtorch|         2137|             2005.0|     102190.0|          17293.0|
|00000c289a1829a80...|           die Ärzte|         1099|             1139.0|     102190.0|            274.0|
|00000c289a1829a80...|   melissa etheridge|          897|              908.0|     102190.0|           2950.0|
|00000c289a1829a80...|           elvenking|          717|              719.0|     102190.0|           1966.0|
|00000c289a1829a80...|juliette & the licks|          706|              706.0|     102190.0|           5285.0|
|00000c289a1829a80...|red hot chili pep...|          691|              697.0|     102190.0|              3.0|
|00000c289

In [ ]:
(training, test) = data.randomSplit([0.5, 0.5])

## Model Building

In [ ]:
USERID = 'User_id_index'
ARTIST = 'Artist_name_index'
COUNT = 'Plays_by_user_index'

als = ALS(maxIter=5, regParam=0.01, userCol=USERID, itemCol=ARTIST, ratingCol=COUNT)
model = als.fit(training)

predictions = model.transform(test)

## Generate top 5 Artist Recommendations for each user

In [ ]:
recs = model.recommendForAllUsers(5)

In [ ]:
#recs.show()

In [ ]:
recs.take(1)

[Row(User_id_index=148, recommendations=[Row(Artist_name_index=13701, rating=4258.02978515625), Row(Artist_name_index=38177, rating=2738.568359375), Row(Artist_name_index=32007, rating=2303.109375), Row(Artist_name_index=16959, rating=2192.452880859375), Row(Artist_name_index=28672, rating=2160.048095703125)])]